# Japan Economic Dashboard

Retrieve data on macroeconomic conditions in Japan and save as csv and txt files to be read by a .tex dashboard.`

In [47]:
import pandas as pd
import requests

In [48]:
# Functions

def write_txt(filename, filetext):
# Write label to txt file
    with open(filename, 'w') as text_file:
        text_file.write(filetext)

### Consumer Prices

In [49]:
# CPI data from e-stat.go.jp
#http://www.e-stat.go.jp/SG1/estat/CsvdlE.do?sinfid=000031431696
cpi_url = 'http://www.e-stat.go.jp/SG1/estat/CsvdlE.do?sinfid=000031431696'
cpi_comp = {
    'All items': 'All',
    'All items, less food (less alcoholic beverages) and energy': 'Core',
    'Food': 'Food \& beverage',
    'Housing': 'Housing',
    'Furniture & household utensils': 'Furniture \& utensils',
    'Clothes & footwear': 'Apparel',
    'Medical care': 'Healthcare',
    'Transportation & communication': 'Transportation',
    'Education': 'Education',
    'Culture & recreation': 'Recreation',
    'Energy': 'Energy'
}

df = pd.read_csv(cpi_url, header=1).loc[424:].set_index('Group/Item')
df = df[cpi_comp.keys()].pct_change(12).dropna()
df['date'] = pd.to_datetime(df.index, format='%Y%m')
df = df.reset_index().set_index('date').drop('Group/Item', 1).multiply(100).round(1)
df.columns = df.columns.to_series().map(cpi_comp)

In [50]:
df[['All', 'Core']].to_csv('data/cpi.csv', header=True)  # csv file created

# Declare variables for chart label
v1 = df['All'].iloc[-1]
v2 = df['Core'].iloc[-1]
d1 = df.index[-1].strftime('%b %Y')
text = '{}: All-items CPI: {}\%; Core CPI: {}\%'.format(d1, v1, v2)
# Write label to txt file
write_txt('data/cpi.txt', text)

In [51]:
# Write legend months to txt file
with open('data/cpi_mo1.txt', 'w') as text_file:
    text_file.write(df.index[-2].strftime('%b %Y'))
with open('data/cpi_mo2.txt', 'w') as text_file:
    text_file.write(df.index[-1].strftime('%b %Y'))  

d = {s: df[s][-120:].mean() for s in df.keys()} # Five year average
    
df = df.drop('Core', 1).tail(2).transpose().round(decimals=1)
df.columns = ['one', 'two']
df.index.name = 'Item'
df['ten'] = df.index.to_series().map(d).round(1)
df = df.set_index(df.reset_index()['Item'].replace('All', '\\textbf{All-items}'))
df = df.sort_values(by='two', axis=0, ascending=False)
df.to_csv('data/cpi_comp.csv', header = True)  # csv file created

### GDP Growth by Component

In [52]:
# GDP
r = requests.get('http://www.esri.cao.go.jp/en/sna/sokuhou/sokuhou_top.html')
sub = r.content[r.text.find('Time series table') - 49:r.text.find('Time series table') - 2]
base = 'http://www.esri.cao.go.jp/en/sna'
base2 = 'http://www.esri.cao.go.jp/jp/sna'
r2 = requests.get(base + sub.replace('..', ''))
rloc = r2.text.find('Real, Seasonally Adjusted Series (Quarter-to-Quarter, Annualized)')
sub2 = r2.content[rloc - 110:rloc - 18]
url = '{}{}'.format(base2, sub2.replace('na/', '/'))
url2 = url.replace('nritu','nkiyo')
url3 = url.replace('nritu-j', 'gaku-m')
url4 = url.replace('nritu', 'gaku')
url5 = url.replace('nritu', 'kgaku')
df = pd.read_csv(url2, header=5).loc[49:]
gdpkeep = {
    'Unnamed: 0': 'date',
    'GDP(Expenditure Approach)': 'gdp',
    'PrivateConsumption': 'cons',
    'PrivateResidentialInvestment': 'inv1',
    'Private Non-Resi.Investment': 'inv2',
    'Changein PrivateInventories': 'inv3',
    'GovernmentConsumption': 'gov1',
    'PublicInvestment': 'gov2',
    'Changein PublicInventories': 'gov3',
    'Goods & Services': 'nx'
}
df = df[gdpkeep.keys()].dropna()
df.columns = df.columns.to_series().map(gdpkeep)

In [53]:
dts = df['date'].str.split('-').str[0].str.split('/ ')
for dt in dts:
    if len(dt) == 1:
        dt.append(dt[0])
        dt[0] = None
df['year'] = dts.str[0].fillna(method='ffill')
df['month'] = dts.str[1].str.zfill(2)
df['date2'] = df['year'].str.cat(df['month'], sep='-')
df['date'] = pd.to_datetime(df['date2'], format='%Y-%m')
df['inv'] = df['inv1'] + df['inv2'] + df['inv3']
df['gov'] = df['gov1'] + df['gov2'] + df['gov3']
df = df.set_index('date')[['gdp', 'cons', 'inv', 'gov', 'nx']]
df.to_csv('data/gdp.csv', header=True)  # csv file created

# Declare variables for chart label
q = df.index[-1].quarter
y = df.index[-1].strftime('%Y')
Y = pd.read_csv(url4, header=5).iloc[49:, 1:2].dropna().iloc[-1].values[0]
ch = df['gdp'][-1]
text = '{} Q{}: Real GDP: {}billion Yen; Growth: {}\%'.format(y, q, Y, ch)
# Write label to txt file
write_txt('data/gdp.txt', text)

### Labor Markets

In [54]:
# Unemployment and participation
unemp = 'http://www.stat.go.jp/data/roudou/longtime/zuhyou/lt01-a10.xls'

df = pd.read_excel(unemp, skiprows=5, skip_footer=3).drop([1, 3])
col1 = df.loc[0].fillna(method='ffill')
col2 = df.loc[2]
col = col1 + '-' + col2
col[0] = 'year'
col[1] = 'mon'
col[2] = 'monname'
col[3] = 'DEL'
df = df.drop([0,2])
df.columns = col.values
df = df.drop('DEL', 1)
df['month'] = df['mon'].str[:-1].str.zfill(2)
df['year'] = df['year'].apply(pd.to_numeric, errors='coerce').shift(-1).fillna(method='ffill').astype(int)
df['date2'] = df['year'].astype(str).str.cat(df['month'], sep='-')
df['date'] = pd.to_datetime(df['date2'], format='%Y-%m')
df = df.set_index('date').dropna()
df['pop'] = df['Labour force-Both sexes'] + df['Not in labour force-Both sexes']
df['partc'] = df['Labour force-Both sexes'] / df['pop'] * 100
df['pop-m'] = df['Labour force-Male'] + df['Not in labour force-Male']
df['partc-m'] = df['Labour force-Male'] / df['pop-m'] * 100
df['pop-w'] = df['Labour force-Female'] + df['Not in labour force-Female']
df['partc-w'] = df['Labour force-Female'] / df['pop-w'] * 100
unemplt = df['Unemployment rate  (percent)-Both sexes'][-1]
df['unemp'] = df['Unemployment rate  (percent)-Both sexes']
d1 = df.index[-1].strftime('%b %Y')
df = df[['partc', 'partc-m', 'partc-w', 'unemp']][636:]

In [55]:
df.to_csv('data/labor.csv', header=True)  # csv file created
text = 'Unemployment rate, both sexes: {}: {}\%'.format(d1, unemplt)
# Write label to txt file
write_txt('data/labor.txt', text)

### Nikkei 225

In [56]:
# Nikkei 225
df = pd.DataFrame()
nikurl = 'http://indexes.nikkei.co.jp/en/nkave/historical/nikkei_stock_average_daily_en.csv'
df['close'] = pd.read_csv(nikurl, skip_footer=1, parse_dates=['Date of Data']).set_index('Date of Data')['Close']
df['vol'] = df['close'].pct_change() * 100
df.index.name = 'date'
d1 = df.index[-1].strftime('%b %#d, %Y')
n1 = '{0:,.1f}'.format(df['close'][-1])

# LaTeX arrows
upar = ' \color{green!80!blue}$\\blacktriangle$\\normalcolor'
dnar = ' \color{red!80!orange}$\\blacktriangledown$\\normalcolor'

d = {}
d['onemo'] = df['close'].dropna().pct_change(periods=21).multiply(100).round(1)[-1]
d['oneyr'] = df['close'].dropna().pct_change(periods=252).multiply(100).round(1)[-1]
for s in ['onemo', 'oneyr']:  # Loop for yearly and monthly changes
    if d[s] > 0.005:
        d[s+'_ar'] = upar # Green up arrow if positive
    elif d[s] < -0.005:
        d[s+'_ar'] = dnar # Red down arrow if negative
    else:
        d[s+'_ar'] = '' # For cases with no change

C:\Users\BDew\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [57]:
df.dropna().to_csv('data/nikkei.csv', header=True)  # csv file created
text = 'One-month change:{} {}\%; \\\ One-year change:{} {}\%'.format(
            d['onemo_ar'], d['onemo'], d['oneyr_ar'], d['oneyr'])
text2 = 'Latest value: {}: {}'.format(d1, n1)
# Write label to txt file
write_txt('data/nikkei.txt', text)
write_txt('data/nikkei2.txt', text2)

### TANKAN

In [58]:
# Tankan Business Conditions
series = {'COCOAEF1000601GCQ00000AT': 'All',
     'COCOAEF1000601GCQ01000AT': 'Large',
     'COCOAEF1000601GCQ02000AT': 'Medium',
     'COCOAEF1000601GCQ03000AT': 'Small'
     }
d = {}
df = pd.DataFrame()
for k, v in series.items():
    url = 'https://www.quandl.com/api/v3/datasets/BOJ/{}.csv?api_key=x7q1kgMKv96cXx83GtSN'.format(k)
    d[v] = pd.read_csv(url, index_col='Date')
df = pd.concat(d, axis=1, ignore_index=True)
df.columns = ['All', 'Large', 'Medium', 'Small']
df = df.dropna()[64:]
df.index = pd.to_datetime(df.index)
d1 = '{} Q{}'.format(df.index[-1].year, df.index[-1].quarter)
all1 = df['All'][-1].astype(int)
df.index.name = 'date'
df.to_csv('data/tankan_bc.csv', header=True)  # csv file created
text = 'As of {}: All-firms: {}'.format(d1, all1)
# Write label to txt file
write_txt('data/tankan_bc.txt', text)

### Industrial Production

In [59]:
# Industrial production
url = 'http://www.meti.go.jp/english/statistics/tyo/iip/xls/b2010_gsm1e.xls'
series = {
    2000000000: '\\textbf{All manufacturing and mining}',
    '2AC0000000': 'Fabricated metals',
    '2AD0000000': 'Machinery',
    '2AE0000000': 'Electrical parts \& devices',
    '2AF0000000': 'Electrical machinery',
    '2AG0000000': 'Information \& communication eq.',
    '2AH0000000': 'Transport equipment',
    '2AJ0000000': 'Chemicals',
    '2AL0000000': 'Plastic products',
    '2AO0000000': 'Foods and tobacco',
    '2AP0000000': 'Other manufacturing',
}

df = pd.read_excel(url, skiprows=2).set_index('Item_Number').ix[:,-60:]
df = df[df.index.isin(series.keys())]
df.replace({'-': None}, regex=True, inplace=True)
dfs = pd.DataFrame()
if len([col for col in df.columns if 'p' in str(col)]) > 0:
    dfs['pre'] = df.iloc[:,-1]
    dfs['lt'] = df.iloc[:,-2]
    predate = '{} (Preliminary)'.format(pd.to_datetime(df.columns[-1][-6:], format='%Y%m').strftime('%b %Y'))
    ltdate = '{}'.format(pd.to_datetime(df.columns[-2], format='%Y%m').strftime('%b %Y'))
    legend = '\legend{{\scriptsize 5-year range, \scriptsize 1-year average, \scriptsize {}, \scriptsize {}}}'.format(ltdate, predate)
else:
    dfs['pre'] = None
    dfs['lt'] = df.iloc[:,-1]  
    predate = None
    ltdate = '{}'.format(pd.to_datetime(df.columns[-1], format='%Y%m').strftime('%b %Y'))
    legend = '\legend{{\scriptsize 5-year range, \scriptsize 1-year avg, \scriptsize {}}}'.format(ltdate)
dfs['avg'] = df.mean(axis=1) #.ix[:,-12:]
dfs['min'] = df.min(axis=1)
dfs['max'] = df.max(axis=1) - dfs['min']
dfs['industry'] = dfs.index.to_series().map(series)
dfs = dfs.sort_values('lt', ascending=False)
dfs = dfs.reset_index()
dfs = dfs.drop('Item_Number', 1).set_index('industry')

In [60]:
write_txt('data/ipleg.txt', legend)
dfs.to_csv('data/ip.csv', header = True)  # csv file created

### Current Account

In [61]:
# Current account from Ministry of Finance balance of payments data
url = 'http://www.mof.go.jp/international_policy/reference/balance_of_payments/bp_trend/bpnet/sbp/s-a/6s-a-1.csv'
columns = ['year' ,'quarter', 'cab', 'gs', 'goods', 'ex', 'im', 'serv', 'income', 'transfers']
series = ['cab', 'goods', 'serv', 'income', 'transfers']
df = pd.read_csv(url, skiprows=28, thousands=',').ix[:,2:].drop('Unnamed: 4',1)
df.columns = columns
df['quarter'] = df['quarter'].str[0]
df['year'] = df['year'].fillna(method='ffill').astype(int)
df['date'] = pd.to_datetime(df['year'].map(str) + 'Q' + df['quarter'])
df = df.set_index('date')
df = df.iloc[40:,:]

In [62]:
# Nominal GDP from cabinet office
ngdp = pd.read_csv(url3, header=5, thousands=',').iloc[49:, 0:2].dropna()
dts = ngdp['Unnamed: 0'].str.split('-').str[0].str.split('/ ')
for dt in dts:
    if len(dt) == 1:
        dt.append(dt[0])
        dt[0] = None
ngdp['year'] = dts.str[0].fillna(method='ffill')
ngdp['month'] = dts.str[1].str.zfill(2)
ngdp['date2'] = ngdp['year'].str.cat(ngdp['month'], sep='-')
ngdp['date'] = pd.to_datetime(ngdp['date2'], format='%Y-%m')
ngdp = ngdp.set_index('date')['GDP(Expenditure Approach)']
ngdp.name = 'gdp'

In [63]:
df = df.join(ngdp).dropna()
df = df[series].div(df['gdp'].multiply(.025), axis=0).round(2).join(df[['ex', 'im']].div(10))
df.to_csv('data/external.csv', header=True)  # csv file created

# Declare variables for chart label
q = df.index[-1].quarter
y = df.index[-1].strftime('%Y')
lt = df['cab'][-1]
exim = 'Exports: {:,}; Imports {:,}'.format(df['ex'][-1], df['im'][-1])
text = '{} Q{}: Current Account Balance: {}\% of GDP'.format(y, q, lt)
text2 = '{} Q{}: {} (goods only, billion Yen)'.format(y, q, exim)
# Write label to txt file
write_txt('data/cab.txt', text)
write_txt('data/tb.txt', text2)

### Investment

In [64]:
df = pd.read_csv(url4, header=5, thousands=',').iloc[49:, [0,5,6,9]].dropna().set_index('Unnamed: 0')
df['tot'] = df.sum(axis=1)

series2 = {'PrivateResidentialInvestment': 'res',
           'Private Non-Resi.Investment': 'bus', 
           'PublicInvestment': 'pub', 
           'tot': 'total'}

df.columns = series2.values()
# Convert to share of total change
for k, v in series2.items():
    df[v+'_ch'] = ((df[v] - df[v].shift(1))/df['total'].shift(1) * 400).round(2)
    
df['quarter'] = df.index.str.split('-').str[0].str.split('/ ').str[-1]
df['year'] = [x for x in df.index.str.split('/ ').str[0].values if len(x) == 4 for n in range(4)][:len(df)]
df['date'] = pd.to_datetime(df['year'].str.cat(df['quarter'], sep='-'))

df = df.reset_index().set_index('date').drop(['quarter', 'year', 'Unnamed: 0'],1).dropna()
df.to_csv('data/inv.csv', header=True)  # csv file created

# Declare variables for chart label
q = df.index[-1].quarter
y = df.index[-1].strftime('%Y')
I = df['total'][-1]
ch = df['total_ch'][-1]
text = 'Total investment expenditure: \\\{} Q{}: {:,} billion Yen; Growth: {}\%'.format(y, q, I, ch)
# Write label to txt file
write_txt('data/inv.txt', text)

### Consumption

In [65]:
# Household consumption expenditures
df = pd.read_csv(url5, header=5, thousands=',').iloc[49:, [0,5,6,7,8]].dropna().set_index('Unnamed: 0')
df['tot'] = df.sum(axis=1)

series = {'DurableGoods': 'durable',
           'Semi-DurableGoods': 'semidur', 
           'Non-DurableGoods': 'nondur', 
           'Services': 'services',
          'tot': 'total'}

df.columns = series.values()
# Convert to share of total change
for k, v in series.items():
    df[v+'_ch'] = ((df[v] - df[v].shift(1))/df['total'].shift(1) * 400).round(2)

df = df.dropna()    
df['quarter'] = df.index.str.split('-').str[0].str.split('/ ').str[-1]
df['year'] = [x for x in df.index.str.split('/ ').str[0].values if len(x) == 4 for n in range(4)][:len(df)]
df['date'] = pd.to_datetime(df['year'].str.cat(df['quarter'], sep='-'))

df = df.reset_index().set_index('date').drop(['quarter', 'year', 'Unnamed: 0'],1).dropna()
df.to_csv('data/cons.csv', header=True)  # csv file created

# Declare variables for chart label
q = df.index[-1].quarter
y = df.index[-1].strftime('%Y')
C = df['total'][-1]
ch = df['total_ch'][-1]
text = 'Total household consumption expenditure: \\\ {} Q{}: {:,} billion Yen; Growth: {}\%'.format(y, q, C, ch)
# Write label to txt file
write_txt('data/cons.txt', text)

### JGBs

In [66]:
url = 'http://www.mof.go.jp/english/jgbs/reference/interest_rate/jgbcme.csv'
url1 = 'http://www.mof.go.jp/english/jgbs/reference/interest_rate/historical/jgbcme_all.csv'

df = pd.read_csv(url, skiprows=1, parse_dates=['Date']).set_index('Date')
df1 = pd.read_csv(url1, skiprows=1, parse_dates=['Date']).iloc[9700:].set_index('Date')

df = df1.append(df)[['1Y', '2Y', '5Y', '10Y', '20Y', '30Y', '40Y']]

prev1 = df.iloc[-246]
prev1dt = '{}'.format(prev1.name.strftime('%b %#d, %Y'))
prev5 = df.iloc[-1227]
prev5dt = '{}'.format(prev5.name.strftime('%b %#d, %Y'))
curr = df.iloc[-1]
currdt = '{}'.format(curr.name.strftime('%b %#d, %Y'))
head = [curr, prev1, prev5]
df1 = pd.concat(head, 1)
df1.columns = ['curr', 'prev1', 'prev5']
df1['number'] = ['1', '2', '3', '4', '5', '6', '7']
df1['alignment'] = 270
df1.to_csv('data/jgb.csv', header=True)  # csv file created

# Write label to txt file
write_txt('data/jgb_prev1.txt', prev1dt)
write_txt('data/jgb_prev5.txt', prev5dt)
write_txt('data/jgb_curr.txt', currdt)

### USD-JPY

In [67]:
url = 'https://www.quandl.com/api/v3/datasets/CURRFX/JPYUSD.csv?api_key=x7q1kgMKv96cXx83GtSN&start_date=2014-01-01'
url2 = 'https://www.quandl.com/api/v3/datasets/BOJ/STFX180110002.csv?api_key=x7q1kgMKv96cXx83GtSN&start_date=2014-01-01'
url3 = 'https://www.quandl.com/api/v3/datasets/BOJ/STFX180110001.csv?api_key=x7q1kgMKv96cXx83GtSN&start_date=2014-01-01'
df1 = pd.read_csv(url, parse_dates=['Date'], index_col='Date')['Rate']
df2 = pd.read_csv(url2, parse_dates=['Date'], index_col='Date')
df3 = pd.read_csv(url3, parse_dates=['Date'], index_col='Date')
df = pd.concat([df1, df2, df3], 1)
df.columns = ['Rate', 'REER', 'NEER']
label = 'As of {}: {} JPY per 1 USD'.format(df.index[-1].strftime('%b %#d, %Y'), (1 / df['Rate'][-1]).round(2))
for column in df.columns:
    df[column] = df[column].dropna() / df[column].dropna()[0]

df.to_csv('data/fx.csv', header=True)  # csv file created

# Write label to txt file
write_txt('data/usdjpy.txt', label)

### Government debt

In [68]:
url4 = 'https://www.quandl.com/api/v3/datasets/BOJ/PFPFGD1.csv?api_key=x7q1kgMKv96cXx83GtSN&start_date=2006-01-01'
url5 = 'https://www.quandl.com/api/v3/datasets/BOJ/PFPFGD220.csv?api_key=x7q1kgMKv96cXx83GtSN&start_date=2006-01-01'
url6 = 'https://www.quandl.com/api/v3/datasets/BOJ/PFPFGD210.csv?api_key=x7q1kgMKv96cXx83GtSN&start_date=2006-01-01'
df1 = pd.read_csv(url4, parse_dates=['Date'], index_col='Date')
df2 = pd.read_csv(url5, parse_dates=['Date'], index_col='Date')
df3 = pd.read_csv(url6, parse_dates=['Date'], index_col='Date')
df = pd.concat([df1, df2, df3], 1)
df.columns = ['Total', 'BOJ', 'Gov']
df = df.iloc[::-1]
df['BOJshare'] = df['BOJ'].div(df['Total']) * 100
df['Govshare'] = df['Gov'].div(df['Total']) * 100
df['Non-BOJ'] = df['Total'] - df['BOJ']
label = 'BOJ share of total: {}: {}\%'.format(df.index[-1].strftime('%b %d, %Y'), round(df['BOJshare'][-1],1))
df.to_csv('data/gov_debt.csv', header=True)  # csv file created

# Write label to txt file
write_txt('data/gov_debt.txt', label)